# Regional Sales Analytics

Combining Graph Traversal with Columnar Aggregation.


In [ ]:
import os
import shutil
import sys

# Ensure we can import the uni module
# Adjust this path if your notebook is in a different location
sys.path.append(os.path.abspath(".."))

import uni


In [ ]:
db_path = "./sales_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)
db = uni.Database(db_path)
print(f"Opened database at {db_path}")


## 1. Schema


In [ ]:
db.create_label("Region")
db.create_label("Order")
db.create_edge_type("SHIPPED_TO", ["Order"], ["Region"])
db.add_property("Region", "name", "string", False)
db.add_property("Order", "amount", "float64", False)


## 2. Ingest Data
One region, 100 orders.


In [ ]:
vids_region = db.bulk_insert_vertices("Region", [{'name': 'North'}])
north = vids_region[0]

orders = [{'amount': 10.0 * (i + 1)} for i in range(100)]
vids_orders = db.bulk_insert_vertices("Order", orders)

edges = [(v, north, {}) for v in vids_orders]
db.bulk_insert_edges("SHIPPED_TO", edges)
db.flush()


## 3. Analytical Query
Sum of amounts for orders in a region.


In [ ]:
query = "MATCH (r:Region {name: 'North'})<-[:SHIPPED_TO]-(o:Order) RETURN SUM(o.amount) as total"
results = db.query(query)
print(f"Total Sales for North Region: {results[0]['total']}")
